# Mandatory Exercise - Session 8

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is to:
-Consider the following sentence: "Lazy cats play with mice."
-Expand the grammar of the example related to non-probabilistic chart parsers in order to subsume this new sentence.
-Perform the constituency parsing using a BottomUpChartParser, a BottomUpLeftCornerChartParser and a LeftCornerChartParser.
-For each one of them, provide the resulting tree, the number of edges and the list of explored edges.
-Which parser is the most efficient for parsing the sentence?
-Which edges are filtered out by each parser and why?

## 1. Constituency parsing with NLTK
This section will define a grammar and use different parsing methods to determine the constituency tree of a sentence.

### 1.1. Grammar definition
The following cell defines a grammar that will allow to parse sentences with very specific terms and relations between them.

In [1]:
import nltk
from nltk import CFG, ChartParser

'''the grammar is expanded considering that:
    - sentences are composed by noun and verb prhases
    - "mice" is added as a possible plural name (NNS)
    - "with" is added as a possible preposition (CC)
    - "play" is added as the only possible verb (V)
    - the verb phrase (VP) is defined as a verb (V) or a verb plus a prepositional phrase (V PP)
    - the prepositional phrase is defined as a preposition plus a noun phrase (CC NP)
'''
grammar = CFG.fromstring('''
                        S -> NP VP
                        NP -> NNS | JJ NNS | NP CC NP 
                        NNS -> "cats" | "dogs" | "mice" | NNS CC NNS
                        JJ -> "big" | "small" | "lazy"
                        CC -> "and" | "or" | "with"
                        V -> "play"
                        VP -> V PP | V
                        PP -> CC NP
                        ''')

# sentence tokens
sentence = nltk.word_tokenize("Lazy cats play with mice".lower())

In [2]:
print(grammar)
print(sentence)

Grammar with 18 productions (start state = S)
    S -> NP VP
    NP -> NNS
    NP -> JJ NNS
    NP -> NP CC NP
    NNS -> 'cats'
    NNS -> 'dogs'
    NNS -> 'mice'
    NNS -> NNS CC NNS
    JJ -> 'big'
    JJ -> 'small'
    JJ -> 'lazy'
    CC -> 'and'
    CC -> 'or'
    CC -> 'with'
    V -> 'play'
    VP -> V PP
    VP -> V
    PP -> CC NP
['lazy', 'cats', 'play', 'with', 'mice']


### 1.2. Constituency parsing
The following cells perform the constituency parsing using a BottomUpChartParser, a BottomUpLeftCornerChartParser and a LeftCornerChartParser.m

In [36]:
from nltk import BottomUpChartParser, ChartParser, LeftCornerChartParser

total_edges = []

print("Sentence to parse: {}\n".format(sentence))

# using different parser classes
parser_type = [(BottomUpChartParser, "BottomUpChartParser"), (ChartParser, "BottomUpLeftCornerChartParser"), (LeftCornerChartParser, "LeftCornerChartParser")]

# using the grammar to create a parser, then parse the sentence with it 
for parser_class, parser_name in  parser_type:    
    parser = parser_class(grammar)
    parsed_sentence = parser.parse(sentence)

    print("Parsing with: {}".format(parser_name))
    
    # showing the constituency trees
    possible_trees = []
    for tree in parsed_sentence:
        possible_trees.append(tree)
    print("Number of trees: ", len(possible_trees))
    for tree in possible_trees:
        print("Parsed tree: ", tree)
        
    # list of the applied edges
    parse = parser.chart_parse(sentence)
    print("Number of edges: {}".format(parse.num_edges()))
    
    edges = parse.edges()
    for edge in edges:
        print(edge)
    if parser_class == BottomUpChartParser:
        total_edges = edges
    else:
        print("\n")
        print("Edges that were filtered out:")
        for edge in total_edges:
            if edge not in edges:
                print(edge)
    print("_________________________________________\n")

Sentence to parse: ['lazy', 'cats', 'play', 'with', 'mice']

Parsing with: BottomUpChartParser
Number of trees:  1
Parsed tree:  (S
  (NP (JJ lazy) (NNS cats))
  (VP (V play) (PP (CC with) (NP (NNS mice)))))
Number of edges: 50
[0:1] 'lazy'
[1:2] 'cats'
[2:3] 'play'
[3:4] 'with'
[4:5] 'mice'
[0:0] JJ -> * 'lazy'
[0:1] JJ -> 'lazy' *
[0:0] NP -> * JJ NNS
[0:1] NP -> JJ * NNS
[1:1] NNS -> * 'cats'
[1:2] NNS -> 'cats' *
[1:1] NP -> * NNS
[1:1] NNS -> * NNS CC NNS
[0:2] NP -> JJ NNS *
[1:2] NP -> NNS *
[1:2] NNS -> NNS * CC NNS
[1:1] S  -> * NP VP
[1:1] NP -> * NP CC NP
[1:2] S  -> NP * VP
[1:2] NP -> NP * CC NP
[0:0] S  -> * NP VP
[0:0] NP -> * NP CC NP
[0:2] S  -> NP * VP
[0:2] NP -> NP * CC NP
[2:2] V  -> * 'play'
[2:3] V  -> 'play' *
[2:2] VP -> * V PP
[2:2] VP -> * V
[2:3] VP -> V * PP
[2:3] VP -> V *
[1:3] S  -> NP VP *
[0:3] S  -> NP VP *
[3:3] CC -> * 'with'
[3:4] CC -> 'with' *
[3:3] PP -> * CC NP
[3:4] PP -> CC * NP
[4:4] NNS -> * 'mice'
[4:5] NNS -> 'mice' *
[4:4] NP -> * NNS
[4

## Discussion
- (A) Which parser is the most efficient for parsing the sentence?
- (B) Which edges are filtered out by each parser and why?